# Results for decision tree classifier
## Multilabel classification with imbalanced data

In [ ]:
"""Results for decision tree classifier

Multilabel classification with imbalanced data
"""

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report

# import data
df = pd.read_csv('data/SCADA_downtime_merged.csv', skip_blank_lines=True)

# list of turbines to plot
list1 = list(df['turbine_id'].unique())
# sort turbines in ascending order
list1 = sorted(list1, key=int)
# list of categories
list2 = list(df['TurbineCategory_id'].unique())
# remove NaN from list
list2 = [g for g in list2 if g >= 0]
# sort categories in ascending order
list2 = sorted(list2, key=int)
# categories to remove
list2 = [m for m in list2 if m not in (1, 12, 13, 14, 15, 17, 21, 22)]
list4 = list(range(0, 14))

# filter only data for turbine x
for x in list1:
    dfx = df[(df['turbine_id'] == x)].copy()
    # copying fault to new column (mins) (fault when turbine category id is y)
    for y in list2:
        def ff(c):
            if c['TurbineCategory_id'] == y:
                return 0
            else:
                return 1
        dfx['mins'] = dfx.apply(ff, axis=1)

        # sort values by timestamp in descending order
        dfx = dfx.sort_values(by='timestamp', ascending=False)
        # reset index
        dfx.reset_index(drop=True, inplace=True)

        # assigning value to first cell if it's not 0
        if dfx.loc[0, 'mins'] == 0:
            dfx.set_value(0, 'mins', 0)
        else:
            dfx.set_value(0, 'mins', 999999999)

        # using previous value's row to evaluate time
        for i, e in enumerate(dfx['mins']):
            if e == 1:
                dfx.at[i, 'mins'] = dfx.at[i-1, 'mins'] + 10

        # sort in ascending order
        dfx = dfx.sort_values(by='timestamp')
        # reset index
        dfx.reset_index(drop=True, inplace=True)
        # convert to hours, then round to nearest hour
        dfx['hours'] = dfx['mins'].astype(np.int64)
        dfx['hours'] = dfx['hours']/60
        dfx['hours'] = round(dfx['hours']).astype(np.int64)

        # > 48 hours - label as normal (9999)
        def f11(c):
            if c['hours'] > 48:
                return 9999
            else:
                return c['hours']
        dfx['hours'] = dfx.apply(f11, axis=1)

        # filter out curtailment - curtailed when turbine is pitching
        # outside 0 deg <= normal <= 3.5 deg
        def f22(c):
            if 0 <= c['pitch'] <= 3.5 or c['hours'] != 9999 or (
                    (c['pitch'] > 3.5 or c['pitch'] < 0) and (
                        c['ap_av'] <= (.1 * dfx['ap_av'].max())
                        or c['ap_av'] >= (.9 * dfx['ap_av'].max()))):
                return 'normal'
            else:
                return 'curtailed'
        dfx['curtailment'] = dfx.apply(f22, axis=1)

        # filter unusual readings, i.e., for normal operation, power <= 0 in
        # operating wind speeds, power > 100...
        # before cut-in, runtime < 600 and other downtime categories
        def f3(c):
            if c['hours'] == 9999 and ((
                        3 < c['ws_av'] < 25 and (
                            c['ap_av'] <= 0 or c['runtime'] < 600 or
                            c['EnvironmentalCategory_id'] > 1 or
                            c['GridCategory_id'] > 1 or
                            c['InfrastructureCategory_id'] > 1 or
                            c['AvailabilityCategory_id'] == 2 or
                            12 <= c['TurbineCategory_id'] <= 15 or
                            21 <= c['TurbineCategory_id'] <= 22)) or
                    (c['ws_av'] < 3 and c['ap_av'] > 100)):
                return 'unusual'
            else:
                return 'normal'
        dfx['unusual'] = dfx.apply(f3, axis=1)

        # round to 6 hour intervals
        def f4(c):
            if c['hours'] == 0:
                return 10
            elif 1 <= c['hours'] <= 6:
                return 11
            elif 7 <= c['hours'] <= 12:
                return 12
            elif 13 <= c['hours'] <= 18:
                return 13
            elif 19 <= c['hours'] <= 24:
                return 14
            elif 25 <= c['hours'] <= 30:
                return 15
            elif 31 <= c['hours'] <= 36:
                return 16
            elif 37 <= c['hours'] <= 42:
                return 17
            elif 43 <= c['hours'] <= 48:
                return 18
            else:
                return 19
        dfx['hours6'] = dfx.apply(f4, axis=1)

        # change label for unusual and curtailed data (20)
        def f5(c):
            if c['unusual'] == 'unusual' or c['curtailment'] == 'curtailed':
                return 20
            else:
                return c['hours6']
        dfx['hours_%s' % y] = dfx.apply(f5, axis=1)

        # drop unnecessary columns
        dfx = dfx.drop('hours6', axis=1)
        dfx = dfx.drop('hours', axis=1)
        dfx = dfx.drop('mins', axis=1)
        dfx = dfx.drop('curtailment', axis=1)
        dfx = dfx.drop('unusual', axis=1)

    # separate features from classes for classification
    features = [
        'ap_av', 'ws_av', 'wd_av', 'pitch', 'ap_max', 'ap_dev',
        'reactive_power', 'rs_av', 'gen_sp', 'nac_pos']
    classes = [col for col in dfx.columns if 'hours' in col]
    # list of columns to copy into new df
    list6 = features + classes
    df2 = dfx[list6].copy()
    # drop NaNs
    df2 = df2.dropna()
    X = df2[features]
    # normalise features to values b/w 0 and 1
    X = preprocessing.normalize(X)
    Y = df2[classes]
    # convert from pd dataframe to np array
    Y = Y.as_matrix()
    # cross validation using time series split
    tscv = TimeSeriesSplit(n_splits=5)

    dt = DecisionTreeClassifier(criterion='entropy')
    # looping for each cross validation fold
    for train_index, test_index in tscv.split(X):
        # split train and test sets
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        # fit to classifier and predict
        dt1 = dt.fit(X_train, Y_train)
        Yp = dt1.predict(X_test)
        for m in list4:
            Yt = Y_test[:, m]
            Ypr = Yp[:, m]
            print(
                'Classification report for turbine %s, turbine category %s'
                % (x, m))
            print(classification_report(Yt, Ypr, digits=6))
        print('-------------------------------------------------------------')

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.988177  0.983815  0.985991     16311
         20   0.763441  0.816092  0.788889      1044

avg / total   0.974658  0.973725  0.974134     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         5
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.970318  0.983644  0.976936     16019
         20   0.763441  0.816874  0.789254      1043

avg / total   0.941504  0.957015  0

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.978140  0.991574  0.984811     15072
         20   0.938825  0.853701  0.894242      2283

avg / total   0.972968  0.973437  0.972897     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       249
         11   0.000000  0.000000  0.000000       192
         12   0.000000  0.000000  0.000000       180
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       180
         15   0.160000  0.025806  0.044444       155
         16   0.000000  0.000000  0.000000       144
         17   0.000000  0.000000  0.000000       144
         18   0.000000  0.000000  0.000000       144
         19   0.926600  0.981767  0.953386     14260
         20   0.642582  0.873608  0.740494      1527

avg / total   0.819322  0.883780  0

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.988891  0.989837  0.989364     13579
         20   0.963327  0.960011  0.961666      3776

avg / total   0.983329  0.983348  0.983337     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        84
         11   0.016260  0.055556  0.025157        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.027778  0.027778  0.027778        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.969362  0.963966  0.966657     13293
         20   0.941710  0.928184  0.934898      3690

avg / total   0.942796  0.935869  0

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.983598  0.989500  0.986540     14000
         20   0.955060  0.931148  0.942952      3355

avg / total   0.978081  0.978220  0.978114     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.037037  0.117647  0.056338        17
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.000000  0.000000        72
         13   0.000000  0.000000  0.000000        72
         14   0.000000  0.000000  0.000000        72
         15   0.000000  0.000000  0.000000        72
         16   0.000000  0.000000  0.000000        72
         17   0.027027  0.013889  0.018349        72
         18   0.000000  0.000000  0.000000        72
         19   0.957642  0.970929  0.964240     13622
         20   0.892701  0.911465  0.901986      3140

avg / total   0.913320  0.927168  0

...

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         19   0.991755  0.987260  0.989503     16327
         20   0.765502  0.835178  0.798824       813

avg / total   0.981023  0.980047  0.980458     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

       10.0   0.000000  0.000000  0.000000         0
       11.0   0.000000  0.000000  0.000000         0
       12.0   0.000000  0.000000  0.000000         0
       13.0   0.000000  0.000000  0.000000         0
       14.0   0.000000  0.000000  0.000000         0
       15.0   0.000000  0.000000  0.000000         0
       16.0   0.000000  0.000000  0.000000         0
       17.0   0.000000  0.000000  0.000000         0
       18.0   0.000000  0.000000  0.000000         0
       19.0   0.991697  0.980217  0.985923     16327
       20.0   0.764172  0.829028  0.795280       813

avg / total   0.980904  0.973046 

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         19   0.979481  0.993559  0.986470     14750
         20   0.956382  0.871548  0.911996      2390

avg / total   0.976260  0.976546  0.976086     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

       10.0   0.000000  0.000000  0.000000         0
       11.0   0.000000  0.000000  0.000000         0
       12.0   0.000000  0.000000  0.000000         0
       13.0   0.000000  0.000000  0.000000         0
       14.0   0.000000  0.000000  0.000000         0
       15.0   0.000000  0.000000  0.000000         0
       16.0   0.000000  0.000000  0.000000         0
       17.0   0.000000  0.000000  0.000000         0
       18.0   0.000000  0.000000  0.000000         0
       19.0   0.979372  0.984949  0.982153     14750
       20.0   0.956804  0.861925  0.906890      2390

avg / total   0.976225  0.967795 

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         19   0.989799  0.991916  0.990856     14967
         20   0.943484  0.929590  0.936486      2173

avg / total   0.983928  0.984014  0.983963     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       521
         11   0.000000  0.000000  0.000000       214
         12   0.000000  0.000000  0.000000       179
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       166
         15   0.000000  0.000000  0.000000       105
         16   0.000000  0.000000  0.000000        78
         17   0.000000  0.000000  0.000000        71
         18   0.000000  0.000000  0.000000        71
         19   0.937316  0.985966  0.961026     14180
         20   0.590426  0.888000  0.709265      1375

avg / total   0.822810  0.886931 

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        40
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        35
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        31
         18   0.000000  0.000000  0.000000        35
         19   0.978793  0.994264  0.986468     15690
         20   0.858569  0.914083  0.885457      1129

avg / total   0.952543  0.970362  0.961339     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

       10.0   0.000000  0.000000  0.000000         0
       11.0   0.000000  0.000000  0.000000         0
       12.0   0.000000  0.000000  


------------------------------------------------------------------------
